![banner](./logo/banner.png)

# Archivos

El manejo de archivos en Python es algo nativo, es decir, por default te da una utileria para comenzar a trabajar con ellos. Esto significa que podemos abrir y leer el contenido de un archivo plano de una manera muy sencilla y rapida.


---
Realizado por Docente: [Alejandro Leyva](https://www.alejandro-leyva.com/)